In [1]:
import pandas as pd

In [2]:
# lendo dados do cartão corporativo do Governo Federal (extrato de janeiro de 2025)
# disponível em: https://portaldatransparencia.gov.br/download-de-dados/cpgf
df = pd.read_csv("202501_CPGF.csv", encoding="latin1", sep=";")

In [3]:
# axis 0 se refere às linhas, axis 1 às colunas
filtered_df = df.drop(["CÓDIGO ÓRGÃO SUPERIOR", 
                       "CÓDIGO ÓRGÃO",
                       "CÓDIGO UNIDADE GESTORA",
                       "ANO EXTRATO",
                       "MÊS EXTRATO"], axis=1)

In [4]:
# Convertendo alguns tipos de dados 

# 'dayfirst=True' significa que no formato de data passado o dia vem antes do mês
# 'errors=coerce' converte valores inválidos em NaT (Not a Time)
filtered_df["DATA TRANSAÇÃO"] = pd.to_datetime(df["DATA TRANSAÇÃO"], dayfirst=True, errors="coerce")

# 'regex=False' "diz" ao pandas que a vírgula não deve ser interpretada como regex, e sim como valor literal
filtered_df["VALOR TRANSAÇÃO"] = (
    df["VALOR TRANSAÇÃO"]
    .str.replace(",", ".", regex=False)
    .astype(float)
)

filtered_df["CNPJ OU CPF FAVORECIDO"] = df["CNPJ OU CPF FAVORECIDO"].astype(str)

In [6]:
filtered_df.head()

,NOME ÓRGÃO SUPERIOR,NOME ÓRGÃO,NOME UNIDADE GESTORA,CPF PORTADOR,NOME PORTADOR,CNPJ OU CPF FAVORECIDO,NOME FAVORECIDO,TRANSAÇÃO,DATA TRANSAÇÃO,VALOR TRANSAÇÃO
0,Advocacia-Geral da União,Advocacia-Geral da União - Unidades com víncul...,SUPERINTENDENCIA REG. DE ADMIN. DA 1ª REGIAO,***.725.752-**,VIVIANE CORREA LIMA,33311039000189,ELETRUS COMERCIO DE MATERIAIS DE CONSTRUCAO LTDA,COMPRA A/V - R$ - APRES,2024-12-12,50.34
1,Advocacia-Geral da União,Advocacia-Geral da União - Unidades com víncul...,SUPERINTENDENCIA REG. DE ADMIN. DA 1ª REGIAO,***.384.652-**,CARLOS EPAMINONDAS GOMES DA SILVA,8632253000270,CROI COMPUTADORES LTDA,COMPRA A/V - R$ - APRES,2024-11-27,575.00
2,Advocacia-Geral da União,Advocacia-Geral da União - Unidades com víncul...,SUPERINTENDENCIA REG. DE ADMIN. DA 1ª REGIAO,***.562.861-**,ANTONIO CARLOS MELO DOS SANTOS,77385797000117,CIBREL COMERCIAL BRASILEIRA DE REFRIGERACAO LI...,COMPRA A/V - R$ - APRES,2024-12-02,183.90
3,Advocacia-Geral da União,Advocacia-Geral da União - Unidades com víncul...,SUPERINTENDENCIA REG. DE ADMIN. DA 1ª REGIAO,***.945.361-**,CLAUDELI CONCEICAO DOS SANTOS,18727053000174,PAGAR.ME INSTITUICAO DE PAGAMENTO S.A,COMPRA A/V - R$ - APRES,2024-11-26,205.00
4,Advocacia-Geral da União,Advocacia-Geral da União - Unidades com víncul...,SUPERINTENDENCIA REG. DE ADMIN. DA 1ª REGIAO,***.945.361-**,CLAUDELI CONCEICAO DOS SANTOS,55930862000105,SOLUCAO ELETRICA LTDA,COMPRA A/V - R$ - APRES,2024-12-05,1750.00


In [8]:
grouped_df = filtered_df.groupby("NOME FAVORECIDO")["VALOR TRANSAÇÃO"]

# desvio padrão baixo significa que os gastos foram mais regulares, em relação a média aritmética
# desvio padrão alto significa que os gastos foram menos regulares, em relação a média aritmética, possivelmente com picos para baixo ou para cima
# mediana divide o conjunto de dados em 2
stats_df = pd.DataFrame({
    "somatorio": grouped_df.sum(),
    "media": grouped_df.mean(),
    "mediana": grouped_df.median(),
    "desvio_padrao": grouped_df.std()
}).reset_index()

In [10]:
stats_df.sort_values(by="mediana", ascending=False)

,NOME FAVORECIDO,somatorio,media,mediana,desvio_padrao
3256,RIBEIRO & RIBEIRO MANUTENCAO E COMERCIO DE PRO...,27322.00,27322.00,27322.00,NaN
1741,H. S. PIRES LTDA,19000.00,19000.00,19000.00,NaN
1467,F. DE A. L. DAMASCENO,17832.00,17832.00,17832.00,NaN
3569,SUPERMERCADO JF LTDA,14000.00,14000.00,14000.00,NaN
2116,L. BEZERRA DA COSTA,12100.00,12100.00,12100.00,NaN
...,...,...,...,...,...
1999,JOSE ALCEU DE SOUZA,5.00,5.00,5.00,NaN
512,BANCO DO ESTADO DO RIO GRANDE DO SUL SA,4.92,4.92,4.92,NaN
1507,FCASTRO MATERIAIS DE CONSTRUCAO LTDA,4.60,4.60,4.60,NaN
2787,PANIFICADORA OLIVEIRA LTDA,2.50,2.50,2.50,NaN


### Teste t

Lembrando que o Teste T (baseado na Distribuição T de Student) assume que a distribuição dos dados se aproxima da Distribuição Normal, ou seja, caso isso
não se aplique talvez não seja o teste de hipótese mais indicado.

1- Extração de amostra estratificada

2- H0 (verificar se a média da amostra em relação a coluna "VALOR TRANSAÇÃO" é maior que 500 R$)

3- H1 (verificar se a média da amostra em relação a coluna "VALOR TRANSAÇÃO" é igual ou menor que 500 R$) 


In [11]:
from scipy import stats

In [12]:
# amostra estratificada por "NOME ÓRGÃO", com 10% de cada grupo
# 'group_keys=False' é para não incluir a chave do grupo (coluna usada no group by) no índice do resultado final
# 'random_state' é a semente do gerador de números aleatórios, que garante a reprodutibilidade do resultado 
amostra_estr = filtered_df.groupby("NOME ÓRGÃO", group_keys=False).sample(frac=0.1, random_state=42)

In [13]:
# O H0 do teste T se refere a hipótese que eu desejo refutar (média <= 500)
# O H1 do teste T se refere a hipótese que rejeita a hipótese nula (média > 500)

# média da hipótese nula
media_hipotese = 500

# removendo valores ausentes (se houver)
valores_amostra = amostra_estr["VALOR TRANSAÇÃO"].dropna()

In [14]:
# Teste t unilateral 
# 'pop_mean' é a média populacional hipotética
# esse método retorna o p_valor BILATERAL, mas eu quero um teste UNILATERAL (média amostral > 500)
t_stat, p_valor_bilateral = stats.ttest_1samp(valores_amostra, popmean=media_hipotese)

In [15]:
# t_stat se refere ao valor T na distribuição t de Student
if t_stat > 0:
    p_valor_unilateral = p_valor_bilateral / 2
else:
    p_valor_unilateral = 1 - (p_valor_bilateral / 2)

In [16]:
print(f"T estatístico: {t_stat}")
print(f"P-valor unilateral (média > 500): {p_valor_unilateral}")

T estatístico: 4.7670023418325025
P-valor unilateral (média > 500): 1.0684546683968389e-06


In [17]:
# O H0 do teste t se refere a hipótese que eu desejo REFUTAR (média <= 500)
# O H1 do teste t se refere a hipótese que rejeita a hipótese nula (média > 500)

# teste com o nível de significância (a=0.01)
alpha = 0.01
if p_valor_unilateral < alpha:
    print("Rejeita H0: diferença estatisticamente significativa")
    print("Portanto, há evidências de que a média amostral é maior que a média assumida, de 500R$")
else:
    print("Não rejeita H0: não há evidência suficiente")
    print("Portanto, não há evidências de que a média amostral é maior que a média assumida, de 500R$")

Rejeita H0: diferença estatisticamente significativa
Portanto, há evidências de que a média amostral é maior que a média assumida, de 500R$
